# Hello, FABRIC: Create your first FABRIC slice

## Configure the Environment

Set the environment variable that will be used by this notebook. If you are using the FABRIC, JupyterHub some of the environment will be automatically configured for you.  You will only need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  If you do not have a bastion username and private key, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).

In [1]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token_beta.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='pruth_0031379841'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/FABRIC/pruth-bastion-1'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Setup the Experiment

#### Import the FABRIC API

In [2]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

#### Create the FABRIC Proxies

The FABRIC API is used via proxy objects that manage connections to the control framework.  

#### (Optional) Query Available Resources

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [ ]:
try:
    available_resources = fablib.get_available_resources()
    print(f"Available Resources: {available_resources}")
    available_resources.draw()
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

## Create the Experiment Slice

#### Configure the Experiment Parameters



In [3]:
slice_name_base = 'MySlice'
sites = [ 'UKY', 'RENC', 'LBNL' ]
node_name_base = 'node'
image = 'default_centos_8'
cores = 2
ram = 8
disk = 10

node_count=10

### Create Slice

<img src="./figs/SingleNode.png" width="20%"><br>

Create a single node with basic compute capabilities. The submit function will block until the node is ready and will display a progress bar.


In [4]:
for site in sites:
    try:
        slice_name=f"{slice_name_base}_{site}"
        print(f"Creating slice {slice_name} at {site}")
        #Create Slice
        slice = fablib.new_slice(slice_name)

        for node_num in range(node_count):
            node_name = f"{node_name_base}{node_num}"
            # Add node
            node = slice.add_node(name=node_name, site=site)
            node.set_capacities(cores=cores, ram=ram, disk=disk)
            node.set_image(image)

        #Submit Slice Request
        slice.submit(wait_progress=False)
    except Exception as e:
        print(f"Slice Failed: {e}")
        traceback.print_exc()
        
        

Creating slice MySlice_UKY at UKY
Creating slice MySlice_RENC at RENC
Creating slice MySlice_LBNL at LBNL


In [6]:
for site in sites:
    try:
        slice_name=f"{slice_name_base}_{site}"
        print(f"Waiting for slice {slice_name} at {site}")

        #Get Slice
        slice = fablib.get_slice(slice_name)

        slice.wait(progress=True)
        #Submit 
        #slice.submit(wait_progress=False)
        slice.post_boot_config(verbose=True)
    except Exception as e:
        print(f"Slice Failed: {e}")
        traceback.print_exc()

Waiting for slice MySlice_UKY at UKY
Waiting for slice . Slice state: StableOK
Waiting for slice MySlice_RENC at RENC
Waiting for slice . Slice state: StableOK
Waiting for slice MySlice_LBNL at LBNL
Waiting for slice . Slice state: StableOK


### Get the New Slice

Your new slice is in the list of all your slices. You can get the specific slice using the fablib library.

In [8]:
for site in sites:
    try:
        slice_name=f"{slice_name_base}_{site}"
        print(f"Getting {slice_name} slice at {site}")
        
        slice = fablib.get_slice(slice_name)
        print(f"Slice: {slice.get_name()}, {slice.get_state()}")
    except Exception as e:
        print(f"Get Slices Fail: {e}")

Getting 'hello, world' slice at UKY
Slice: MySlice_UKY, StableOK
Getting 'hello, world' slice at RENC
Slice: MySlice_RENC, StableOK
Getting 'hello, world' slice at LBNL
Slice: MySlice_LBNL, StableOK


### Print the Node's Attributes

Each node in the slice has a set of get functions that return the node's attributes.

In [9]:
for site in sites:
    try:
        slice_name=f"{slice_name_base}_{site}"

        print(f"Get attributes for {slice_name} at {site}")    
        slice = fablib.get_slice(slice_name)
        for node in slice.get_nodes():
            print("Node:")
            print(f"   Name              : {node.get_name()}")
            print(f"   Cores             : {node.get_cores()}")
            print(f"   RAM               : {node.get_ram()}")
            print(f"   Disk              : {node.get_disk()}")
            print(f"   Image             : {node.get_image()}")
            print(f"   Image Type        : {node.get_image_type()}")
            print(f"   Host              : {node.get_host()}")
            print(f"   Site              : {node.get_site()}")
            print(f"   Management IP     : {node.get_management_ip()}")
            print(f"   Reservation ID    : {node.get_reservation_id()}")
            print(f"   Reservation State : {node.get_reservation_state()}")
            print(f"   Components        : {node.get_components()}")
            print(f"   Interfaces        : {node.get_interfaces()}")
            print(f"   SSH Command       : {node.get_ssh_command()}")
            print()    
    except Exception as e:
        print(f"Fail: {e}")

Get attributes for MySlice_UKY at UKY
Node:
   Name              : node0
   Cores             : 2
   RAM               : 8
   Disk              : 10
   Image             : default_centos_8
   Image Type        : qcow2
   Host              : uky-w1.fabric-testbed.net
   Site              : UKY
   Management IP     : 128.163.179.37
   Reservation ID    : d925e0df-81bb-4530-8ecb-0bacb0c7df07
   Reservation State : Active
   Components        : []
   Interfaces        : []
   SSH Command       : ssh -i /Users/pruth/.ssh/id_rsa -J pruth_0031379841@bastion-1.fabric-testbed.net centos@128.163.179.37

Node:
   Name              : node1
   Cores             : 2
   RAM               : 8
   Disk              : 10
   Image             : default_centos_8
   Image Type        : qcow2
   Host              : uky-w1.fabric-testbed.net
   Site              : UKY
   Management IP     : 128.163.179.32
   Reservation ID    : 4ea6af95-b0d2-41d6-8afa-3b83791b9f41
   Reservation State : Active
   Components  

## Accessing the VM

#### Terminal SSH Command

You can access the nodes in you experiments using ssh. In order to increase security, FABRIC requires all ssh connections to jump through one of the FABRIC bastion hosts. The bastion host IPs and your bastion keys should have been set at the top of this notebook. 

The ssh command required to access your node from this Jupyter instance can be found using the following library call.  Note that using this command from a different computer, such as your laptop, may require you to copy ssh keys to that computer and use different paths to the keys. 


In [ ]:
for site in sites:
    try:
        slice_name=f"{slice_name_base}_{site}"

        print(f"Getting ssh command for {slice_name} at {site}")    
        slice = fablib.get_slice(slice_name)
        for node in slice.get_nodes():
            print(f"   Name              : {node.get_name()}")
            print(f"   SSH Command       : {node.get_ssh_command()}")
            print()    
    except Exception as e:
        print(f"Fail: {e}")

#### SSH with fablib

Most experiments will require automated configuration and execution. You can use the fablib library to execute arbitrary commands on your node. 

The following code demonstrates how to use fablib to execute a "Hello, FABRIC" bash script. The library uses the bastion and VM keys defined at the top of this notebook to jump through the bastion host and execute the script.

In [10]:
command= 'echo Hello, FABRIC from node `hostname -s`'

In [11]:
for site in sites:
    try:
        slice_name=f"{slice_name_base}_{site}"

        print(f"Executing command for {slice_name} at {site}")    
        slice = fablib.get_slice(slice_name)
        for node in slice.get_nodes():
            stdout, stderr = node.execute(command)
            print(stdout)
    except Exception as e:
        print(f"Fail: {e}")

Executing command for MySlice_UKY at UKY
Hello, FABRIC from node d925e0df-81bb-4530-8ecb-0bacb0c7df07-node0

Hello, FABRIC from node 4ea6af95-b0d2-41d6-8afa-3b83791b9f41-node1

Hello, FABRIC from node a538b271-952e-44fe-9a93-922a856e8203-node2

Hello, FABRIC from node 6e150d58-a1c5-4d11-8974-7bbbe24980d2-node3

Hello, FABRIC from node 31acb94f-92f0-485e-99a7-917df5e17091-node4

Hello, FABRIC from node 143b327a-46a1-42c0-a02e-d10f21509262-node5

Hello, FABRIC from node d69db24e-0f3a-461c-91f0-74d20c4c9cea-node6

Hello, FABRIC from node 4f59cf99-f90e-4efe-93a2-255321a52f65-node7

Hello, FABRIC from node fb375370-022f-4301-b4e1-680f88663ef9-node8

Hello, FABRIC from node a98232bb-db13-475c-90f1-f0f5e2e915a0-node9

Executing command for MySlice_RENC at RENC
Hello, FABRIC from node 1bbfa088-543c-4ef9-a7c1-47783c361011-node0

Hello, FABRIC from node d356aea7-c05f-4413-9b52-14a0a8a4985c-node1

Hello, FABRIC from node 90ac6f5c-60f2-413f-bf69-6365a2be3280-node2

Hello, FABRIC from node 1c6b89a6

## Delete Slice

Please delete your slicd when you are done with your experiment.

In [12]:
for site in sites:
    try:
        slice_name=f"{slice_name_base}_{site}"

        print(f"Deleting for {slice_name} at {site}")    
        slice = fablib.get_slice(slice_name)
        slice.delete()
    except Exception as e:
        print(f"Fail: {e}")

Deleting for MySlice_UKY at UKY
Fail: "(invalid_grant) invalid refresh token"

Deleting for MySlice_RENC at RENC
Fail: "(invalid_grant) invalid refresh token"

Deleting for MySlice_LBNL at LBNL
Fail: "(invalid_grant) invalid refresh token"

